In [1]:
import pandas as pd

In [ ]:
!mkdir data
!cd data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet

In [2]:
!pip install pyarrow

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [ ]:
import mlflow
mlflow.set_tracking_uri()

In [6]:
df = pd.read_parquet("./data/green_tripdata_2021-01.parquet")

In [ ]:
df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df

In [ ]:
df.duration = df.duration.apply(lambda td: td.total_seconds()/60) ## changing to total minutes ## .total_seconds is the function of time delta objects
df

In [ ]:
df = df[(df.duration>=1)&(df.duration <=60)]
df.dtypes

In [ ]:
copy_df = df.copy()
categorical = ["PULocationID","DOLocationID"]
numerical = ["trip_distance"]

copy_df[categorical] = copy_df[categorical].astype(str)


In [ ]:
train_dicts = copy_df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = "duration"
y_train = copy_df[target].values


In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_train)

In [ ]:
y_pred

In [ ]:
root_mean_squared_error(y_train, y_pred)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()


In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    copy_df = df.copy()
    categorical = ['PULocationID', 'DOLocationID']
    copy_df[categorical] = copy_df[categorical].astype(str)
    
    return copy_df

In [ ]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
!mkdir models


In [ ]:
!git add --all


In [ ]:
!git commit -m "Finished Setting Up and Training and Storing a Model"

In [ ]:
!git push origin main
